In [2]:
import os
import numpy as np
import json
import cv2

import math
import tensorflow as tf
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_root_dir = '/kaggle/input/shoes-classification-dataset-13k-images/Shoes Dataset/Train'
val_root_dir = "/kaggle/input/shoes-classification-dataset-13k-images/Shoes Dataset/Valid"

datagen = ImageDataGenerator(
    rescale = 1/255,
)

train_generator = datagen.flow_from_directory(
    train_root_dir,
    target_size=(254,254),
    batch_size=64,
    class_mode='categorical',
#     color_mode='grayscale'
)

val_generator = datagen.flow_from_directory(
    val_root_dir,
    target_size=(254,254),
    batch_size=64,
    shuffle=False,
    class_mode='categorical',
#     color_mode='grayscale'
)

Found 10000 images belonging to 5 classes.
Found 2500 images belonging to 5 classes.


In [4]:
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers

model = models.Sequential()

# Input shape: (254, 254, 3)

# Convolutional Block 1
model.add(layers.Conv2D(64, (3, 3), padding='same', input_shape=(254, 254, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))  # Image size: (127, 127, 64)

# Convolutional Block 2
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))  # Image size: (64, 64, 128)

# Convolutional Block 3
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))  # Image size: (32, 32, 256)

# Convolutional Block 4
model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))  # Image size: (16, 16, 512)

# Convolutional Block 5
model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2, 2)))  # Image size: (8, 8, 512)

# Global Average Pooling
model.add(layers.GlobalAveragePooling2D())

# Fully Connected Layers
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

# Output Layer
model.add(layers.Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer=optimizers.Nadam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)
model_checkpoint = callbacks.ModelCheckpoint(filepath='/kaggle/working/model/attr2.keras', monitor='val_accuracy', mode='max', save_best_only=True, verbose=2)

# Train the model
epochs = 30
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[reduce_lr, model_checkpoint],
    batch_size=32
)


Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-08-23 12:44:41.360265: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng15{k5=1,k6=0,k7=1,k10=1} for conv (f32[64,64,254,254]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,254,254]{3,2,1,0}, f32[64,3,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-08-23 12:44:42.168190: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 30: 3.91532, expecte

 31/157 ━━━━━━━━━━━━━━━━━━━━ 1:10 558ms/step - accuracy: 0.2516 - loss: 4.5517

2024-08-23 12:46:39.783505: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.94228, expected 3.22586
2024-08-23 12:46:39.783564: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 4.89184, expected 4.17542
2024-08-23 12:46:39.783573: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 6.10896, expected 5.39254
2024-08-23 12:46:39.783581: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.83507, expected 3.11864
2024-08-23 12:46:39.783589: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 4.99526, expected 4.27884
2024-08-23 12:46:39.783596: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 5.21842, expected 4.502
2024-08-23 12:46:39.783604: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 4.79188, expected 4.07546
2024-08-23 12:46:39.783612: E external/local_xla/xla/serv

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 788ms/step - accuracy: 0.2656 - loss: 4.2467

2024-08-23 12:48:37.294255: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.79125, expected 2.85665
2024-08-23 12:48:37.294317: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 4.36138, expected 3.42678
2024-08-23 12:48:37.294330: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 4.53851, expected 3.6039
2024-08-23 12:48:37.294344: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.39154, expected 2.45694
2024-08-23 12:48:37.294362: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 4.20481, expected 3.27021
2024-08-23 12:48:37.294375: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 4.15202, expected 3.21741
2024-08-23 12:48:37.294386: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 4.0684, expected 3.1338
2024-08-23 12:48:37.294397: E external/local_xla/xla/servi


Epoch 1: val_accuracy improved from -inf to 0.17480, saving model to /kaggle/working/model/attr2.keras
157/157 ━━━━━━━━━━━━━━━━━━━━ 262s 927ms/step - accuracy: 0.2657 - loss: 4.2451 - val_accuracy: 0.1748 - val_loss: 4.4525 - learning_rate: 0.0010
Epoch 2/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 556ms/step - accuracy: 0.3006 - loss: 3.5204
Epoch 2: val_accuracy improved from 0.17480 to 0.28720, saving model to /kaggle/working/model/attr2.keras
157/157 ━━━━━━━━━━━━━━━━━━━━ 97s 609ms/step - accuracy: 0.3007 - loss: 3.5196 - val_accuracy: 0.2872 - val_loss: 3.1691 - learning_rate: 0.0010
Epoch 3/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 556ms/step - accuracy: 0.3486 - loss: 3.0401
Epoch 3: val_accuracy did not improve from 0.28720
157/157 ━━━━━━━━━━━━━━━━━━━━ 96s 602ms/step - accuracy: 0.3485 - loss: 3.0397 - val_accuracy: 0.2136 - val_loss: 3.5478 - learning_rate: 0.0010
Epoch 4/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 556ms/step - accuracy: 0.3475 - loss: 2.6983
Epoch 4: val_accuracy did not improve from

In [5]:
model.save('my_model.h5')